In [25]:
import findspark
findspark.init('C:\spark-3.3.1-bin-hadoop-3')
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local").\
    appName("Word Count").\
    config("spark.driver.BindAdress", "localhost").\
    config("spark.ui.port", "4040").\
    getOrCreate()

In [ ]:
#a.       Создайте схему будущего фрейма данных. Схема должна включать следующие атрибуты:
#·   id -  уникальный идентификатор посетителя сайта. Тип – последовательность чисел фиксированной длины. Данное поле не является первичным ключом.
#·   timestamp – дата и время события в формате unix timestamp.
#·   type – тип события, значение из списка (факт посещения(visit), клик по визуальному элементу страницы(click), скролл(scroll), перед на другую страницу(move)).
#·   page_id – id текущей страницы. Тип - последовательность чисел фиксированной длины.
#·   tag – каждая страница с новостью размечается редакцией специальными тегами, которые отражают тематику конкретной новости со страницы. Возможный список тематик: политика, спорт, медицина и т.д.
#·   sign – наличие у пользователя личного кабинета. Значения – True/False.
#b.       Создайте датафрейм с описанной выше схемой данных.
#c.       Наполните датафрейм данными. Пример:
#(12345, 1667627426, "click", 101, "Sport”, False)

In [70]:
import pyspark.sql.types as T
import pyspark.sql
import pyspark.sql.functions
import datetime
simpleData = ((1, "02-01-2022 11:01:19", "click", 3, "politics", True),
    (2, "03-01-2022 12:01:19","scroll", 4, "sport", True),
    (3, "11-07-2022 12:06:07","move", 4, "medicine", True),
    (4, "11-14-2022 23:06:07", "click", 3, "politics", False),
    (5, "11-21-2022 03:22:58","scroll", 9, "medicine", False),
    (4, "11-30-2022 12:59:41","scroll", 4, "sport", True),
    (3, "01-23-2022 12:54:53","move", 4, "medicine", True),
    (2, "01-16-2022 01:54:53", "click", 3, "politics", False),
    (1, "01-09-2022 09:08:02","scroll", 9, "medicine", False))
schema = T.StructType([ \
    T.StructField("id", T.IntegerType(), True), \
    T.StructField("timestamp", T.StringType(),True), \
    T.StructField("type", T.StringType(),True), \
    T.StructField("page_id", T.IntegerType(), True), \
    T.StructField("tag", T.StringType(), True), \
    T.StructField("sign", T.BooleanType(), True) \
])
df = spark.createDataFrame(spark.sparkContext.parallelize(simpleData),schema)
df1=df.select(col("id"), unix_timestamp("timestamp", "MM-dd-yyyy HH:mm:ss").alias("timestamp"), col("type"),col("page_id"), col("tag"), col("sign"))
df1.printSchema()
df1.show()

root
 |-- id: integer (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- type: string (nullable = true)
 |-- page_id: integer (nullable = true)
 |-- tag: string (nullable = true)
 |-- sign: boolean (nullable = true)

+---+----------+------+-------+--------+-----+
| id| timestamp|  type|page_id|     tag| sign|
+---+----------+------+-------+--------+-----+
|  1|1643702479| click|      3|politics| true|
|  2|1646125279|scroll|      4|   sport| true|
|  3|1667811967|  move|      4|medicine| true|
|  4|1668456367| click|      3|politics|false|
|  5|1668990178|scroll|      9|medicine|false|
|  4|1669802381|scroll|      4|   sport| true|
|  3|1642931693|  move|      4|medicine| true|
|  2|1642287293| click|      3|politics|false|
|  1|1641708482|scroll|      9|medicine|false|
+---+----------+------+-------+--------+-----+



In [88]:
#·   Вывести топ-5 самых активных посетителей сайта
df2=df1.groupby("id").count().orderBy(col("id")).show()

+---+-----+
| id|count|
+---+-----+
|  1|    2|
|  2|    2|
|  3|    2|
|  4|    2|
|  5|    1|
+---+-----+



In [152]:
#·   Посчитать процент посетителей, у которых есть ЛК
p=df1.filter(col("sign")==True).count()/df1.count()*100
p

55.55555555555556

In [139]:
#·   Вывести топ-5 страниц сайта по показателю общего кол-ва кликов на данной странице
df3=df1.filter(col("type")=="click").groupby("page_id").count().orderBy(col("page_id")).show()


+-------+-----+
|page_id|count|
+-------+-----+
|      3|    3|
+-------+-----+



In [143]:
#·   Добавьте столбец к фрейму данных со значением временного диапазона в рамках суток с размером окна – 4 часа(0-4, 4-8, 8-12 и т.д.)
from pyspark.sql.functions import when, lit, col
df4=df1.withColumn("timed", when(((hour(from_unixtime("timestamp")))>=0) & ((hour(from_unixtime("timestamp")))<=4), lit("0-4"))
    .when(((hour(from_unixtime("timestamp")))>=4) & ((hour(from_unixtime("timestamp")))<=8), lit("4-8"))
    .when(((hour(from_unixtime("timestamp")))>=8) & ((hour(from_unixtime("timestamp")))<=12), lit("8-12"))
    .when(((hour(from_unixtime("timestamp")))>=12) & ((hour(from_unixtime("timestamp")))<=16), lit("12-16"))
    .when(((hour(from_unixtime("timestamp")))>=16) & ((hour(from_unixtime("timestamp")))<=20), lit("16-20"))
    .otherwise(lit("20-24")))
df4.show()

+---+----------+------+-------+--------+-----+-----+
| id| timestamp|  type|page_id|     tag| sign|timed|
+---+----------+------+-------+--------+-----+-----+
|  1|1643702479| click|      3|politics| true| 8-12|
|  2|1646125279|scroll|      4|   sport| true| 8-12|
|  3|1667811967|  move|      4|medicine| true| 8-12|
|  4|1668456367| click|      3|politics|false|20-24|
|  5|1668990178|scroll|      9|medicine|false|  0-4|
|  4|1669802381|scroll|      4|   sport| true| 8-12|
|  3|1642931693|  move|      4|medicine| true| 8-12|
|  2|1642287293| click|      3|politics|false|  0-4|
|  1|1641708482|scroll|      9|medicine|false| 8-12|
+---+----------+------+-------+--------+-----+-----+



In [146]:
#·   Выведите временной промежуток на основе предыдущего задания, в течение которого было больше всего активностей на сайте.
df5=df4.groupby("timed").count().orderBy(col("count").desc()).show(1)

+-----+-----+
|timed|count|
+-----+-----+
| 8-12|    6|
+-----+-----+
only showing top 1 row



In [ ]:
#·   Создайте второй фрейм данных, который будет содержать информацию о ЛК посетителя сайта со следующим списком атрибутов
#1.       Id – уникальный идентификатор личного кабинета
#2.       User_id – уникальный идентификатор посетителя
#3.       ФИО посетителя
#4.    Дату рождения посетителя 
#5.       Дата создания ЛК

In [157]:
oneMoreSimpleData = ((1, 1, "Абрамов Илья Борисович", datetime.datetime.strptime("1978-08-03", "%Y-%m-%d").date(), datetime.datetime.strptime("2002-02-28", "%Y-%m-%d").date()),
    (2, 2,"Бажов Александр Викторович", datetime.datetime.strptime("1971-12-27", "%Y-%m-%d").date(), datetime.datetime.strptime("2001-12-03", "%Y-%m-%d").date()),
    (3, 3,"Васильева Оксана Валерьевна", datetime.datetime.strptime("1983-01-20", "%Y-%m-%d").date(), datetime.datetime.strptime("2017-11-10", "%Y-%m-%d").date()),
    (4, 4, "Грешковец Андрей Макарович", datetime.datetime.strptime("1951-05-05", "%Y-%m-%d").date(), datetime.datetime.strptime("2001-07-25", "%Y-%m-%d").date()),
    (5, 5,"Дадыкина Валентина Тимофеевна", datetime.datetime.strptime("1940-11-29", "%Y-%m-%d").date(), datetime.datetime.strptime("2000-01-01", "%Y-%m-%d").date()))
newschema = T.StructType([ \
    T.StructField("id", T.IntegerType(), False), \
    T.StructField("User_id", T.IntegerType(),True), \
    T.StructField("fio", T.StringType(),True), \
    T.StructField("birth", T.DateType(), True), \
    T.StructField("lkdate", T.DateType(), True) \
])
dff = spark.createDataFrame(spark.sparkContext.parallelize(oneMoreSimpleData), newschema)
dff.printSchema()
dff.show()

root
 |-- id: integer (nullable = false)
 |-- User_id: integer (nullable = true)
 |-- fio: string (nullable = true)
 |-- birth: date (nullable = true)
 |-- lkdate: date (nullable = true)

+---+-------+--------------------+----------+----------+
| id|User_id|                 fio|     birth|    lkdate|
+---+-------+--------------------+----------+----------+
|  1|      1|Абрамов Илья Бори...|1978-08-03|2002-02-28|
|  2|      2|Бажов Александр В...|1971-12-27|2001-12-03|
|  3|      3|Васильева Оксана ...|1983-01-20|2017-11-10|
|  4|      4|Грешковец Андрей ...|1951-05-05|2001-07-25|
|  5|      5|Дадыкина Валентин...|1940-11-29|2000-01-01|
+---+-------+--------------------+----------+----------+



In [165]:
#·   Вывести фамилии посетителей, которые читали хотя бы одну новость про спорт.
dfff=dff.join(df1, dff.id==df1.id).filter(col("tag")=="sport").show()

+---+-------+--------------------+----------+----------+---+----------+------+-------+-----+----+
| id|User_id|                 fio|     birth|    lkdate| id| timestamp|  type|page_id|  tag|sign|
+---+-------+--------------------+----------+----------+---+----------+------+-------+-----+----+
|  2|      2|Бажов Александр В...|1971-12-27|2001-12-03|  2|1646125279|scroll|      4|sport|true|
|  4|      4|Грешковец Андрей ...|1951-05-05|2001-07-25|  4|1669802381|scroll|      4|sport|true|
+---+-------+--------------------+----------+----------+---+----------+------+-------+-----+----+

